In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from NoEmbedRNN import OpSel
from NoEmbedRNN import MemSel
from NoEmbedRNN import RNN as oldRNN
from NoEmbedRNN import MemRNN  as oldMemRNN
from NoEmbedRNN import HistoryRNN
from ops import Operations
from session import *
from data_gen import *
import pickle

modelName = 'RNN'
old = True
func = 'np_avg_val-5ops'
name = '/hardmax_one_did_kindof_conv_beforex_embed/state_size_400-num_samples_3500-batch_size_100-learning_rate_0.001-epsilon_0.001-num_features_10-norm_True-softmax_sat_1-clip_False-share_state_True-rnns_same_state_False-state_fn_relu-pen_sofmax_True-smax_pen_r_0.05-seed_47913'
path = '/home/user/neural_program_synthesis/models/summaries/'+modelName+'/'+func+'/'+name+'/'
model_path = path+'/model'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
ops.ops = cfg["used_ops_obj"]
ops.num_of_ops = len(ops.ops)
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if modelName == "RNN":
        #instantiante the mem selection RNN
        if old: mem = oldMemRNN(cfg, ops)
        else:   mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        if old: model = oldRNN(cfg, ops, mem)
        else:   model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "HistoryRNN":
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
[<tf.Variable 'RNN_mem/Params/W_mem:0' shape=(410, 400) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b_mem:0' shape=(400,) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/W2_mem:0' shape=(400, 4) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b2_mem:0' shape=(4,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W:0' shape=(410, 400) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b:0' shape=(400,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W2:0' shape=(400, 4) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b2:0' shape=(4,) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_0:0' shape=(410, 400) dtype=float64>, <tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_1:0' shape=(400,) dtype=float64>, <tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_glob

In [29]:
def print_errors(error_lst, name):
    print("")
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [30]:
def print_matrix(total_error, matrix_lst, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
    print("")

In [31]:
def print_ops_matrix(total_error, matrix_lst, ops_list, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
    print("")

In [32]:
print_errors(res["total_loss_traind_train"], "sofmax_train_error")
print_errors(res["total_loss_traind_test"], "hardmax_train_error")
print_errors(res["total_loss_testd_train"], "softmax_test_error")
print_errors(res["total_loss_testd_test"], "hardmax_test_error")


sofmax_train_error[0] err is 197.370313601
sofmax_train_error[1] err is 442.834519784
sofmax_train_error[2] err is 558.226672335
sofmax_train_error[3] err is 762.676457656
sofmax_train_error[4] err is 362.860043099
sofmax_train_error[5] err is 435.659102549
sofmax_train_error[6] err is 700.769790956
sofmax_train_error[7] err is 945.409872026
sofmax_train_error[8] err is 952.691962022
sofmax_train_error[9] err is 333.159771671
sofmax_train_error[10] err is 500.431859847
sofmax_train_error[11] err is 960.432866558
sofmax_train_error[12] err is 315.927013749
sofmax_train_error[13] err is 553.090452461
sofmax_train_error[14] err is 533.184557227
sofmax_train_error[15] err is 606.257776242
sofmax_train_error[16] err is 1205.17299348
sofmax_train_error[17] err is 778.542309253
sofmax_train_error[18] err is 376.626736004
sofmax_train_error[19] err is 923.44749734
sofmax_train_error[20] err is 1241.18750975
sofmax_train_error[21] err is 1262.84754459
sofmax_train_error[22] err is 987.89432281

In [33]:
#print_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"])

Error for this matrix is 36.8967463502

##Elem-0--#############################################################
[ tf_stall ] [ tf_add ] [ tf_stall ] [ tf_stall ] [ tf_divide ] 
##Elem-1--#############################################################
[ tf_stall ] [ tf_add ] [ tf_stall ] [ tf_stall ] [ tf_divide ] 
##Elem-2--#############################################################
[ tf_stall ] [ tf_add ] [ tf_stall ] [ tf_stall ] [ tf_divide ] 
##Elem-3--#############################################################
[ tf_stall ] [ tf_add ] [ tf_stall ] [ tf_stall ] [ tf_divide ] 
##Elem-4--#############################################################
[ tf_stall ] [ tf_add ] [ tf_stall ] [ tf_stall ] [ tf_divide ] 
##Elem-5--#############################################################
[ tf_stall ] [ tf_add ] [ tf_stall ] [ tf_stall ] [ tf_divide ] 
##Elem-6--#############################################################
[ tf_stall ] [ tf_add ] [ tf_stall ] [ tf_stall ] [ tf_divide ] 
#

In [34]:
#print_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_test"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

Error for this matrix is 36.8967463502

##Elem-0--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-1--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-2--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-3--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-4--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-5--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-6--################################

In [35]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_train"][0])

Error for this matrix is 36.8967463502

##Elem-0--#############################################################
[-45.632 -99.438 -48.212 -68.945 -12.563 -92.873  83.568  75.266 -27.615
  -8.991] [-245.436    0.       0.       0.       0.       0.       0.       0.       0.
    0.   ] [-245.436    0.       0.       0.       0.       0.       0.       0.       0.
    0.   ] [-245.436    0.       0.       0.       0.       0.       0.       0.       0.
    0.   ] [-24.544   0.      0.      0.      0.      0.      0.      0.      0.      0.   ] 
##Elem-1--#############################################################
[-39.458  -1.814  56.755 -52.603 -96.067 -69.963 -34.358 -69.948  52.69
 -79.754] [-334.519    0.       0.       0.       0.       0.       0.       0.       0.
    0.   ] [-334.519    0.       0.       0.       0.       0.       0.       0.       0.
    0.   ] [-334.519    0.       0.       0.       0.       0.       0.       0.       0.
    0.   ] [-33.452   0.      0.      0

In [36]:
wrong_res = np.nonzero(np.round(np.apply_along_axis(np_add, 1, res["outputs_testd_train"][0][4] - res["batchesY_test"][0]), 2))[0]
wrong_res 

array([52, 61, 82, 94])

In [37]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"], wrong_res)

Error for this matrix is 36.8967463502

##Elem-52--#############################################################
[ tf_stall ] [ tf_stall ] [ tf_divide ] [ tf_stall ] [ tf_add ] 
##Elem-61--#############################################################
[ tf_stall ] [ tf_stall ] [ tf_divide ] [ tf_stall ] [ tf_add ] 
##Elem-82--#############################################################
[ tf_stall ] [ tf_stall ] [ tf_divide ] [ tf_stall ] [ tf_add ] 
##Elem-94--#############################################################
[ tf_stall ] [ tf_stall ] [ tf_divide ] [ tf_stall ] [ tf_add ] 


In [38]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"], wrong_res)

Error for this matrix is 36.8967463502

##Elem-52--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-61--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-82--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-94--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 


In [ ]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_test_mem"][0])

In [ ]:
res["batchesX_test"][0]

In [ ]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
res = predict_form_sess(model, cfg, x_test[0:100,], last_hardmax_state_train, model_path, mode="hard")

In [ ]:
res - y_test[0:100,]

In [ ]:
y_test[0:100,]

In [ ]:
res

In [ ]:
len(res["softmaxes_traind_train"])